In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
train = pd.read_csv('train.csv', low_memory=False)
test = pd.read_csv('test.csv', low_memory=False)

In [3]:
train = train.loc[train['text'].drop_duplicates().index]

In [4]:
test['target'] = 2
totaldata = pd.concat([train,test])
totaldata['textsize'] = totaldata['text'].apply(len)
totaldata['haslocacion'] = totaldata['location'].notna()
totaldata.drop('location',axis=1,inplace=True)

In [5]:
############## keyword

In [6]:
keyword = pd.read_csv('data_process/keyword.csv')
totaldata = totaldata.merge(keyword,how='left')
totaldata.drop('keyword',axis=1,inplace=True)
del keyword

In [7]:
############## links, label, mentions

In [8]:
def get_links(x):
    l = re.findall(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',x)
    return l
def hashtags(x): 
    return re.findall(r'#[a-zA-z0-9_]{1,}',x)
def arrobas(x):
    return re.findall(r'@[a-zA-z0-9_]{1,}',x)

totaldata[["mentions_count","links_count","label_count"]] =\
                        totaldata["text"].apply(lambda x: pd.Series([len(arrobas(x)),len(hashtags(x)),len(get_links(x))]))

In [9]:
############# dominos 

In [10]:
enlaces_target = pd.read_csv('data_process/enlaces.csv')
totaldata = totaldata.merge(enlaces_target,how='left').fillna(0)
del enlaces_target

In [11]:
############# taggs

In [12]:
tagged=pd.read_csv('data_process/tagged.csv')
totaldata = pd.concat([totaldata, tagged],axis=1)
totaldata.to_csv('data_process/totaldata.csv',index=False)